## Lesson 17 - Statistics Packages

In this lesson we'll consider some of the various statistics tools available in Python. Many packages provide statistical support: Pandas, Numpy, Scipy, and Scikit-Learn.

You will probably have to install `scikit-learn` (`sklearn`) and `outlier_utils` (`outliers`) before proceeding:

```
conda install scikit-learn
pip install outlier_utils
```

In [1]:
# import required packages
import pandas as pd
import numpy as np
import scipy
from outliers import smirnov_grubbs as grubbs
import skbio
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ImportError: No module named seaborn

In [ ]:
# set up pandas and seaborn environments
pd.set_option('display.max_rows', 25)
sns.set()
sns.set_context('notebook')
sns.set_palette('colorblind')

### Basic statistics

#### Basic stats with Pandas

A large number of methods for computing descriptive statistics and other related operations on Series, DataFrame, and Panel. Most of these are aggregations (hence producing a lower-dimensional result) like `sum()`, `mean()`, and `quantile()`, but some of them, like `cumsum()` and `cumprod()`, produce an object of the same size. Generally speaking, these methods take an axis argument, just like `ndarray.{sum, std, ...}`, but the axis can be specified by name or integer:

* Series: no axis argument needed
* DataFrame: “index” (axis=0, default), “columns” (axis=1)
* Panel: “items” (axis=0), “major” (axis=1, default), “minor” (axis=2)

Function | Description
--------- | ----------
count | Number of non-null observations
sum | Sum of values
mean | Mean of values
mad | Mean absolute deviation
median | Arithmetic median of values
min | Minimum
max | Maximum
mode | Mode
abs | Absolute Value
prod | Product of values
std | Bessel-corrected sample standard deviation
var | Unbiased variance
sem | Standard error of the mean
skew | Sample skewness (3rd moment)
kurt | Sample kurtosis (4th moment)
quantile | Sample quantile (value at %)
cumsum | Cumulative sum
cumprod | Cumulative product
cummax | Cumulative maximum
cummin | Cumulative minimum

#### Example: Monthly precipitation in La Jolla from 2008 to 2016

In [ ]:
# format the data: covert to datetime, average precipitation per month, get month and year, reset index
df = pd.read_csv('../data/la_jolla_precip_monthly.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.groupby('DATE').mean()
df['MONTH'] = [x.month for x in df.index]
df['YEAR'] = [x.year for x in df.index]
df.reset_index(inplace=True)

In [ ]:
# examine the resulting dataframe
df.head()

In [ ]:
# explore the data using a plot
plt.plot(df.DATE, df.PRCP);

In [ ]:
# make sure our index is sequential
plt.plot(df.index, df.PRCP);

In [ ]:
# describe
df.describe()

In [ ]:
# mean
df.PRCP.mean()

In [ ]:
# std
df.PRCP.std()

In [ ]:
# quantile
df.PRCP.quantile(0.25), df.PRCP.quantile(0.5), df.PRCP.quantile(0.75)

In [ ]:
# min
df.PRCP.min(), df.PRCP.idxmin()

In [ ]:
# max (with rounding)
df.PRCP.max(), df.PRCP.max().round(), df.PRCP.idxmax()

In [ ]:
# cumsum
df.PRCP.cumsum()

In [ ]:
# value_counts
df.PRCP.round().value_counts()

### Regression analysis

#### Regression with Seaborn

In [ ]:
sns.regplot(x='MONTH', y='PRCP', data=df, order=1)

In [ ]:
sns.regplot(x='MONTH', y='PRCP', data=df, order=2)

Seaborn is handy to generate plots, but it doesn't provide easy access to the coefficients. For more control, we can use Numpy, Scipy, and other statistics packages.

#### Regression with Numpy

##### np.polyfit - least squares polynomial fit (1st order)

In [ ]:
# 1st order with np.polyfit
m, b = np.polyfit(df.MONTH, df.PRCP, 1)

In [ ]:
# plot scatter and polyfit
plt.scatter(df.MONTH, df.PRCP)
plt.plot(df.MONTH, m*df.MONTH + b, '-');

In [ ]:
# sort DataFrame by month, then re-plot
df.sort_values('MONTH', inplace=True)
plt.scatter(df.MONTH, df.PRCP)
plt.plot(df.MONTH, m*df.MONTH + b, '-');

##### np.polyfit - least squares polynomial fit (2nd order)

In [ ]:
# 2nd order with np.polyfit
p = np.polyfit(df.MONTH, df.PRCP, 2)

In [ ]:
# values of p are in decending orders
p

In [ ]:
# create a finely spaced array
x1 = np.linspace(1,12)
x1

In [ ]:
# calculate the y vector for the fit curve 
y1 = np.polyval(p, x1)
y1

In [ ]:
plt.scatter(df.MONTH, df.PRCP)
plt.plot(x1, y1, '-');

In [ ]:
# 3rd order with np.polyfit
p = np.polyfit(df.MONTH, df.PRCP, 3)

In [ ]:
y1 = np.polyval(p, x1)

In [ ]:
plt.scatter(df.MONTH, df.PRCP)
plt.plot(x1, y1, '-');

#### Fitting time series to a sinusoidal wave

In [ ]:
# sine wave refresher
period = 4*np.pi
freq = (2*np.pi)/period
phase = 0
amplitude = .5
offset = 1
x1 = np.linspace(0, 24, num=2000)
y1 = np.sin(x1 * freq + phase) * amplitude + offset

fig, ax = plt.subplots()
ax.plot(x1, y1)
ax.set_xticks([2*np.pi, 4*np.pi, 6*np.pi])
ax.set_xticklabels(['2$\pi$', '4$\pi$', '6$\pi$']);

In [ ]:
# we are going to use the numerical index for our "t" variable
# make sure our index is sequential (December 2008 is zero)
df.sort_index(inplace=True)
df.head()

In [ ]:
plt.scatter(df.index, df.PRCP)
plt.xticks(np.arange(0, df.index.max(), 12));

In [ ]:
# store our values as new variables
t = df.index
data = df.PRCP

In [ ]:
# guess the sine wave properties
guess_period = 12
guess_freq = (2*np.pi)/guess_period
guess_phase = 0
guess_amplitude = 10
guess_offset = 10

p0 = [guess_freq, guess_amplitude, guess_phase, guess_offset]

In [ ]:
# create the function we want to fit
def my_sin(x, freq, amplitude, phase, offset):
    return np.sin(x * freq + phase) * amplitude + offset

In [ ]:
# now do the fit
fit = scipy.optimize.curve_fit(my_sin, t, data, p0=p0)

In [ ]:
# we'll use this to plot our first estimate. This might already be good enough for you
data_first_guess = my_sin(t, *p0)

In [ ]:
# recreate the fitted curve using the optimized parameters
data_fit = my_sin(t, *fit[0])

In [ ]:
plt.plot(data, '.')
plt.plot(data_fit, label='after fitting')
plt.plot(data_first_guess, label='first guess')
plt.xticks(np.arange(0, df.index.max(), 12))
plt.legend();

#### Regression with Scipy

#### Example: Moons of the Solar System

In [ ]:
df_moons = pd.read_excel('../data/moons.xlsx')
df_planets = pd.read_excel('../data/planets.xlsx')
df_solar = pd.merge(df_moons, df_planets, left_on='planet_name', right_on='planet_name')
df_solar['moon_volume_km3'] = 4/3*np.pi*(df_solar.moon_diameter_km/2)**3
df_solar['planet_volume_km3'] = 4/3*np.pi*(df_solar.planet_diameter_km/2)**3
df_solar

##### Pearson correlation

In [ ]:
# pearson correlation (linear regression on values) of diameters
pearson_r_diameter, pearson_p_diameter = scipy.stats.pearsonr(df_solar.planet_diameter_km, df_solar.moon_diameter_km)
pearson_r_diameter, pearson_p_diameter

In [ ]:
# pearson correlation (linear regression on values) of volumes
pearson_r_volume, pearson_p_volume = scipy.stats.pearsonr(df_solar.planet_volume_km3, df_solar.moon_volume_km3)
pearson_r_volume, pearson_p_volume

##### Spearman correlation

In [ ]:
# spearman correlation (linear regression on ranks) of diameters
spearman_r_diameter, spearman_p_diameter = scipy.stats.spearmanr(df_solar.planet_diameter_km, df_solar.moon_diameter_km)
spearman_r_diameter, spearman_p_diameter

In [ ]:
# spearman correlation (linear regression on ranks) of volumes
spearman_r_volume, spearman_p_volume = scipy.stats.spearmanr(df_solar.planet_volume_km3, df_solar.moon_volume_km3)
spearman_r_volume, spearman_p_volume

In [ ]:
# plot linear regressions
fig, ax = plt.subplots(1, 2, figsize=(10,5))
sns.regplot(x='planet_diameter_km', y='moon_diameter_km', data=df_solar, ax=ax[0])
sns.regplot(x='planet_volume_km3', y='moon_volume_km3', data=df_solar, ax=ax[1]);

What if we consider the *ratio* of moon size to planet size?

In [ ]:
# calculate moon diameters and volumes relative to host planets
df_solar['moon_planet_relative_diameter'] = df_solar.moon_diameter_km/df_solar.planet_diameter_km
df_solar['moon_planet_relative_volume'] = df_solar.moon_volume_km3/df_solar.planet_volume_km3

In [ ]:
# plot scatter plots
fig, ax = plt.subplots(1, 2, figsize=(10,5))

ax[0].scatter(df_solar.moon_diameter_km, df_solar.moon_planet_relative_diameter)
ax[0].set_xlabel('moon_diameter_km')
ax[0].set_ylabel('moon_planet_relative_diameter')

ax[1].scatter(df_solar.moon_volume_km3, df_solar.moon_planet_relative_volume)
ax[1].set_xlabel('moon_volume_km3')
ax[1].set_ylabel('moon_planet_relative_volume')

fig.tight_layout()

In [ ]:
# plot distributions
fig, ax = plt.subplots(1, 2, figsize=(10,5))
sns.distplot(df_solar.moon_planet_relative_diameter, rug=True, bins=20, ax=ax[0])
sns.distplot(df_solar.moon_planet_relative_volume, hist=False, kde=False, rug=True, bins=20, ax=ax[1]);
ax[1].set_xscale('log')

#### Grubbs's test for outliers

Grubbs's test is used to detect outliers in a univariate data set assumed to come from a normally distributed population.

`outlier_utils` is a library for detecting and removing outliers using Grubbs's test.

In [ ]:
# print outliers from the dataset - relative diameter
grubbs.max_test_outliers(df_solar.moon_planet_relative_diameter, alpha=0.05)

In [ ]:
# remove outliers - relative diameter
moon_planet_rel_diam_no_outliers = grubbs.test(df_solar.moon_planet_relative_diameter, alpha=0.05)
moon_planet_rel_diam_no_outliers

In [ ]:
# print outliers from the dataset - relative volume (higher alpha)
grubbs.max_test_outliers(df_solar.moon_planet_relative_volume, alpha=0.000005)

In [ ]:
# remove outliers - relative volume (higher alpha)
moon_planet_rel_vol_no_outliers = grubbs.test(df_solar.moon_planet_relative_volume, alpha=0.000005)
moon_planet_rel_vol_no_outliers

### Tests of indpendence (of two nominal variables)

Source: [Handbook of Biological Statistics](http://www.biostathandbook.com) by John H. McDonald

Test | Purpose | Notes | Example
----- | ----- | ----- | -----
Fisher's exact test | Test hypothesis that proportions are the same in different groups | Use for small sample sizes (less than 1000) | Count the number of live and dead patients after treatment with drug or placebo, test the hypothesis that the proportion of live and dead is the same in the two treatments, total sample <1000
Chi-square test of independence | Test fit of observed frequencies to expected frequencies | Use for large sample sizes (greater than 1000) | Count the number of live and dead patients after treatment with drug or placebo, test the hypothesis that the proportion of live and dead is the same in the two treatments, total sample >1000

#### Fisher's exact test

Use the Fisher's exact test of independence when you have two nominal variables and you want to see whether the proportions of one variable are different depending on the value of the other variable. Use it when the sample size is small.

Parameters
* table: array_like of ints.
    A 2x2 contingency table.  Elements should be non-negative integers.
* alternative: {'two-sided', 'less', 'greater'}, optional.
    Which alternative hypothesis to the null hypothesis the test uses.
    Default is 'two-sided'.

Returns
* oddsratio: float.
    This is prior odds ratio and not a posterior estimate.
* p_value: float.
    P-value, the probability of obtaining a distribution at least as
    extreme as the one that was actually observed, assuming that the
    null hypothesis is true.

Say we spend a few days counting whales and sharks in the Atlantic and
Indian oceans. In the Atlantic ocean we find 8 whales and 1 shark, in the
Indian ocean 2 whales and 5 sharks. Then our contingency table is:

In [ ]:
f = pd.DataFrame([[8, 2],[1, 5]], 
                 index=['Atlantic', 'Indian'], 
                 columns=['whales', 'sharks'])
f

We use this table to find the p-value:

In [ ]:
odds_ratio, p_value = scipy.stats.fisher_exact(f, alternative='two-sided')
odds_ratio, p_value

The probability that we would observe this or an even more imbalanced ratio
by chance is about 3.5%.  A commonly used significance level is 5%--if we
adopt that, we can therefore conclude that our observed imbalance is
statistically significant; whales prefer the Atlantic while sharks prefer
the Indian Ocean.

For tables with large numbers, the (inexact) chi-square test implemented
in the function `chi2_contingency` can also be used.

#### Chi-square test of independence

Use the chi-square test of independence when you have two nominal variables and you want to see whether the proportions of one variable are different for different values of the other variable. Use it when the sample size is large.

Parameters:
* observed: array_like.
    The contingency table. The table contains the observed frequencies
    (i.e. number of occurrences) in each category.  In the two-dimensional
    case, the table is often described as an "R x C table".

Returns:

* chi2: float.
    The test statistic.
* p: float.
    The p-value of the test
* dof: int.
    Degrees of freedom
* expected: ndarray, same shape as `observed`.
    The expected frequencies, based on the marginal sums of the table.

In [ ]:
chi2, p, dof, expected = scipy.stats.chi2_contingency(f)
chi2, p, dof, expected

### Analysis of variance

#### One-sample *t*-test

The one-sample *t*-test is a two-sided test for the null hypothesis that the expected value
(mean) of a sample of independent observations `a` is equal to the given
population mean, `popmean`.

Parameters:
* a: array_like.
    Sample observation
* popmean: float or array_like.
    Expected value in null hypothesis, if array_like than it must have the
    same shape as `a` excluding the axis dimension
    
Returns:
* statistic: float or array.
    t-statistic
* pvalue: float or array.
    Two-tailed p-value

The population mean of moon-to-planet diameters (excluding our Moon) is approximately 0.018. Our moon is excluded from this population because its relative diameter of 0.273 is an outlier.

In [ ]:
moon_planet_rel_diam_no_outliers.mean()

We **cannot** reject the null hypothesis that an expected mean of 0.02 is equal to population mean.

In [ ]:
scipy.stats.ttest_1samp(moon_planet_rel_diam_no_outliers, 0.02)

We **can** reject the null hypothesis that an expected mean equal to the Moon-Earth diamter ratio (0.273) is equal to population mean.

In [ ]:
scipy.stats.ttest_1samp(moon_planet_rel_diam_no_outliers, 0.273)

#### Two-sample *t*-test

The two-sample *t* test is a two-sided test for the null hypothesis that two independent samples
have identical average (expected) values. This test assumes that the populations have identical variances by default.

Parameters
* a, b: array_like.
    The arrays must have the same shape, except in the dimension
    corresponding to `axis` (the first, by default).
* axis : int or None, optional
    Axis along which to compute test. If None, compute over the whole
    arrays, `a`, and `b`.
    
Returns
* statistic: float or array.
    The calculated t-statistic.
* pvalue: float or array.
    The two-tailed p-value.

In [ ]:
# create sample data
np.random.seed(1)
a = np.random.randn(40)
b = 1.5*np.random.randn(50)+2

In [ ]:
# scipy.stats.ttest_ind (t-test for the means of two independent samples)
t, p = scipy.stats.ttest_ind(a, b, equal_var=False)
print("ttest_ind: t = %g, p = %g" % (t, p))

In [ ]:
# compute the descriptive statistics of a and b
abar = a.mean()
avar = a.var(ddof=1)
na = a.size
adof = na - 1

bbar = b.mean()
bvar = b.var(ddof=1)
nb = b.size
bdof = nb - 1

In [ ]:
# scipy.stats.ttest_ind_from_stats (t-test for the means of two independent samples from descriptive stats)
t2, p2 = scipy.stats.ttest_ind_from_stats(abar, np.sqrt(avar), na,
                              bbar, np.sqrt(bvar), nb,
                              equal_var=False)
print("ttest_ind_from_stats: t = %g, p = %g" % (t2, p2))

In [ ]:
# use the formulas directly
tf = (abar - bbar) / np.sqrt(avar/na + bvar/nb)
dof = (avar/na + bvar/nb)**2 / (avar**2/(na**2*adof) + bvar**2/(nb**2*bdof))
pf = 2*scipy.special.stdtr(dof, -np.abs(tf))

In [ ]:
# plot the distributions
sns.distplot(a, bins=10, label='a')
sns.distplot(b, bins=10, label='b')
plt.legend();

#### One-way anova

The one-way ANOVA tests the null hypothesis that two or more groups have
the same population mean.  The test is applied to samples from two or
more groups, possibly with differing sizes.

Parameters
* sample1, sample2, ... : array_like.
    The sample measurements for each group.

Returns
* statistic: float.
    The computed F-value of the test.
* pvalue: float.
    The associated p-value from the F-distribution.

Note: The ANOVA test has important assumptions that must be satisfied in order
for the associated p-value to be valid.

1. The samples are independent.
2. Each sample is from a normally distributed population.
3. The population standard deviations of the groups are all equal.  This
   property is known as homoscedasticity.

If these assumptions are not true for a given set of data, it may still be
possible to use the Kruskal-Wallis H-test (`scipy.stats.kruskal`) although
with some loss of power.

Here are some data on a shell measurement (the length of the anterior
adductor muscle scar, standardized by dividing by length) in the mussel
*Mytilus trossulus* from five locations: Tillamook, Oregon; Newport, Oregon;
Petersburg, Alaska; Magadan, Russia; and Tvarminne, Finland, taken from a
much larger data set used in McDonald et al. (1991).

In [ ]:
tillamook = [0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735, 0.0659, 0.0923, 0.0836]
newport = [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835, 0.0725]
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [ ]:
sns.distplot(tillamook, label='Tillamook')
sns.distplot(newport, label='Newport')
sns.distplot(petersburg, label='Petersburg')
sns.distplot(magadan, label='Magadan')
sns.distplot(tvarminne, label='Tvarminne')
plt.legend();

In [ ]:
scipy.stats.f_oneway(tillamook, newport, petersburg, magadan, tvarminne)

The means **were** significantly heterogeneous (one-way anova, $F_{4,34}=7.12, P=2.8\times10^{-4}$). 

Complete ANOVA results (see http://www.biostathandbook.com/onewayanova.html):

. | sum of squares | d.f. | mean square | Fs | P
-----|-----|-----|-----|-----|-----
among groups | 0.00452| 4 | 0.001113 | 7.12 | 2.8e-4
within groups | 0.00539 | 34 | 0.000159 | | 
total | 0.00991 | 38 |  |  |

### Microbiome analysis

This last lesson is taken from the documentation for [Scikit-bio](http://scikit-bio.org/docs/latest/). The package `skbio.diversity` provides diversity measures for OTU tables. OTUs are "operational taxonomic units"; you can think of them as taxa or species.

In [ ]:
from skbio.diversity import alpha_diversity
from skbio import TreeNode
from io import StringIO
from skbio.diversity import beta_diversity
from skbio.stats.distance import mantel
from skbio.stats.ordination import pcoa
from skbio.stats.distance import anosim

Create a matrix containing 6 samples (rows) and 7 OTUs (columns):

In [ ]:
data = [[23, 64, 14, 0, 0, 3, 1],
        [0, 3, 35, 42, 0, 12, 1],
        [0, 5, 5, 0, 40, 40, 0],
        [44, 35, 9, 0, 1, 0, 0],
        [0, 2, 8, 0, 35, 45, 1],
        [0, 0, 25, 35, 0, 19, 0]]
ids = list('ABCDEF')

In [ ]:
data

In [ ]:
ids

#### Alpha-diversity

First, we’ll compute observed OTUs, an alpha diversity metric, for each sample using the alpha_diversity driver function:

In [ ]:
adiv_obs_otus = alpha_diversity('observed_otus', data, ids)
adiv_obs_otus

Next we’ll compute Faith’s PD on the same samples. Since this is a phylogenetic diversity metric, we’ll first create a tree and an ordered list of OTU identifiers.

In [ ]:
tree = TreeNode.read(StringIO(
               '(((((OTU1:0.5,OTU2:0.5):0.5,OTU3:1.0):1.0):0.0,'
               '(OTU4:0.75,(OTU5:0.5,(OTU6:0.5,OTU7:0.5):0.5):'
               '0.5):1.25):0.0)root;'))
otu_ids = ['OTU1', 'OTU2', 'OTU3', 'OTU4', 'OTU5', 'OTU6', 'OTU7']
adiv_faith_pd = alpha_diversity('faith_pd', data, ids=ids, otu_ids=otu_ids, tree=tree)
adiv_faith_pd

#### Beta-diversity

Now we’ll compute Bray-Curtis distances, a beta diversity metric, between all pairs of samples. Notice that the data and ids parameters provided to beta_diversity are the same as those provided to alpha_diversity.

In [ ]:
bc_dm = beta_diversity("braycurtis", data, ids)
print(bc_dm)

Next, we’ll compute weighted UniFrac distances between all pairs of samples. Because weighted UniFrac is a phylogenetic beta diversity metric, we’ll need to pass the skbio.TreeNode and list of OTU ids that we created above. Again, these are the same values that were provided to alpha_diversity.

In [ ]:
wu_dm = beta_diversity("weighted_unifrac", data, ids, tree=tree, otu_ids=otu_ids)
print(wu_dm)

Next we’ll do some work with these beta diversity distance matrices. First, we’ll determine if the UniFrac and Bray-Curtis distance matrices are significantly correlated by computing the Mantel correlation between them. Then we’ll determine if the p-value is significant based on an alpha of 0.05.

In [ ]:
r, p_value, n = mantel(wu_dm, bc_dm)
print(r)
alpha = 0.05
print(p_value)
print(p_value < alpha)

Next, we’ll perform principal coordinates analysis (PCoA) on our weighted UniFrac distance matrix.

In [ ]:
wu_pc = pcoa(wu_dm)
print(wu_pc)

PCoA plots are only really interesting in the context of sample metadata, so let’s define some before we visualize these results.

In [ ]:
sample_md = [
    ('A', ['gut', 's1']),
    ('B', ['skin', 's1']),
    ('C', ['tongue', 's1']),
    ('D', ['gut', 's2']),
    ('E', ['tongue', 's2']),
    ('F', ['skin', 's2'])]
sample_md = pd.DataFrame.from_items(sample_md, columns=['body_site', 'subject'], orient='index')
sample_md

Now let’s plot our PCoA results, coloring each sample by the subject it was taken from:

In [ ]:
sns.set_style('white')
fig = wu_pc.plot(sample_md, 'subject',
    axis_labels=('PC 1', 'PC 2', 'PC 3'),
    title='Samples colored by subject', cmap='Set1', s=50)

We don’t see any clustering/grouping of samples. If we were to instead color the samples by the body site they were taken from, we see that the samples from the same body site (those that are colored the same) appear to be closer to one another in the 3-D space then they are to samples from other body sites.

In [ ]:
fig = wu_pc.plot(sample_md, 'body_site',
    axis_labels=('PC 1', 'PC 2', 'PC 3'),
    title='Samples colored by body site', cmap='Set1', s=50)

Ordination techniques, such as PCoA, are useful for exploratory analysis. The next step is to quantify the strength of the grouping/clustering that we see in ordination plots. There are many statistical methods available to accomplish this; many operate on distance matrices. Let’s use ANOSIM to quantify the strength of the clustering we see in the ordination plots above, using our weighted UniFrac distance matrix and sample metadata.

First test the grouping of samples by **subject**:

In [ ]:
results = anosim(wu_dm, sample_md, column='subject', permutations=999)

In [ ]:
results

In [ ]:
results['test statistic']

In [ ]:
results['p-value'] < 0.1

The negative value of ANOSIM’s R statistic indicates anti-clustering, but the p-value is insignificant at an alpha of 0.1.

Now let’s test the grouping of samples by **body site**:

In [ ]:
results = anosim(wu_dm, sample_md, column='body_site', permutations=999)

In [ ]:
results

In [ ]:
results['test statistic']

In [ ]:
results['p-value'] < 0.1

The R statistic indicates strong separation of samples based on body site. The p-value is significant at an alpha of 0.1.

#### Correlations with metadata

We can also explore the alpha diversity in the context of sample metadata. To do this, let’s add the Observed OTU and Faith PD data to our sample metadata. This is straight-forward beause alpha_diversity returns a Pandas Series object, and we’re representing our sample metadata in a Pandas DataFrame object.

In [ ]:
sample_md['Observed OTUs'] = adiv_obs_otus
sample_md['Faith PD'] = adiv_faith_pd
sample_md

We can investigate these alpha diversity data in the context of our metadata categories. For example, we can generate boxplots showing Faith PD by body site.

In [ ]:
fig = sample_md.boxplot(column='Faith PD', by='body_site')

We can also compute Spearman correlations between all pairs of columns in this DataFrame. Since our alpha diversity metrics are the only two numeric columns (and thus the only columns for which Spearman correlation is relevant), this will give us a symmetric 2x2 correlation matrix.

In [ ]:
sample_md.corr(method="spearman")

### R kernel in Jupyter notebooks

Finally, if you want to use R in Jupyter notebooks, installation and instructions are here: <https://irkernel.github.io>.